In [1]:
from __future__ import annotations
from HTorch.MCTensor import MCTensor
from HTorch.MCHTensor import MCHTensor, MCHParameter
from HTorch.HTensor import manifold_maps, HParameter, HTensor
import torch
from torch.nn import Parameter
from HTorch.MCTensor import MCTensor
from torch import Tensor
from HTorch.optimizers import RiemannianSGD, RiemannianAdam


c:\Users\23128\anaconda3\envs\venv\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# class MCHParameter(MCHTensor, Parameter):
#     """
#     Subclass of torch.nn.Parameter for Riemannian optimization with MCTensor arithmetic
#     """
#     @staticmethod
#     def __new__(cls, data, manifold='PoincareBall', curvature=-1.0, nc=1, requires_grad=True):
#         res = MCHTensor._make_subclass(cls, data, requires_grad)
#         return res
    
#     def __init__(self, x, manifold='PoincareBall', curvature=-1.0, nc=1, dtype=None, device=None):
#         if isinstance(x, MCHTensor):
#             self.manifold = x.manifold
#             self.curvature = x.curvature
#             self._nc = x._nc
#             self.res = x.res.clone()
#         elif isinstance(x, MCTensor):
#             self.manifold = manifold_maps[manifold]()
#             self.curvature = curvature
#             self._nc = x._nc
#             self.res = x.res.clone()
#         else:
#             self.manifold = manifold_maps[manifold]()
#             self.curvature = curvature
#             self._nc = nc
#             self.res = torch.zeros(x.size() + (nc-1,),
#                               dtype=dtype, device=device)

#     def init_weights(self, irange=1e-5):
#         # this irange need to be controled for different floating-point precision
#         self.data.copy_(self.manifold.init_weights(self, abs(self.curvature), irange))
#         self.res.zero_()

In [3]:
mch_x = MCHTensor([0.2, 0.1, 0.3], manifold='Lorentz', nc=3, curvature=-1)
# mch_x.proj_()
mch_param = MCHParameter(mch_x, manifold='Lorentz')
print('mch_param before updates', mch_param)
torch.autograd.set_detect_anomaly(True)
rsgd_mch = RiemannianSGD([mch_param], lr=1e-2)
# mch_dist = mch_param.Hdist(mch_param.origin(2, 1, ()))
mch_dist = mch_param.norm()
print('mch_dist (loss)', mch_dist)
mch_dist.backward()
print('mch_param.grad', mch_param.grad)
rsgd_mch.step()
print('mch_param after updates', mch_param)
print()

h_x = HTensor([0.2, 0.1, 0.3], manifold='Lorentz', curvature=-1)
h_param = HParameter(h_x, manifold='Lorentz')
rsgd_h = RiemannianSGD([h_param], lr=1e-2)
h_dist = h_param.norm()
h_dist.backward()
print('h_dist (loss)', h_dist)
print('h_param.grad', h_param.grad)
rsgd_h.step()
print('h_param after updates', h_param)


mch_param before updates Parameter containing:
Parameter(MCHParameter([0.2000, 0.1000, 0.3000], requires_grad=True)), nc=3, manifold=Lorentz, curvature=-1
mch_dist (loss) tensor(0.3742, grad_fn=<AliasBackward0>)
mch_param.grad tensor([0.5345, 0.2673, 0.8018])
mch_param after updates Parameter containing:
Parameter(MCHParameter([0.1939, 0.0970, 1.0232], requires_grad=True)), nc=3, manifold=Lorentz, curvature=-1

h_dist (loss) tensor(0.3742, grad_fn=<AliasBackward0>)
h_param.grad tensor([0.5345, 0.2673, 0.8018])
h_param after updates Hyperbolic Parameter containing:
Parameter(HParameter([0.1939, 0.0970, 1.0232], requires_grad=True)), manifold=Lorentz, curvature=-1


In [4]:
mch_x = MCHTensor([0.2, 0.1, 0.3], manifold='Lorentz', nc=3, curvature=-1)
# mch_x.proj_()
mch_param = MCHParameter(mch_x, manifold='Lorentz')
print('mch_param before updates', mch_param)
torch.autograd.set_detect_anomaly(True)
radam_mch = RiemannianAdam([mch_param], lr=1e-1)
# mch_dist = mch_param.Hdist(mch_param.origin(2, 1, ()))
mch_dist = mch_param.dist(mch_param)
print('mch_dist (loss)', mch_dist)
mch_dist.backward()
print('mch_param.grad', mch_param.grad)
radam_mch.step()
print('mch_param after updates', mch_param)
print()

h_x = HTensor([0.2, 0.1, 0.3], manifold='Lorentz', curvature=-1)
h_param = HParameter(h_x, manifold='Lorentz')
radam_h = RiemannianAdam([h_param], lr=1e-1)
h_dist = h_param.dist(h_param)
h_dist.backward()
print('h_dist (loss)', h_dist)
print('h_param.grad', h_param.grad)
radam_h.step()
print('h_param after updates', h_param)


mch_param before updates Parameter containing:
Parameter(MCHParameter([0.2000, 0.1000, 0.3000], requires_grad=True)), nc=3, manifold=Lorentz, curvature=-1
mch_dist (loss) tensor(0., grad_fn=<DistBackward0>)
mch_param.grad tensor([0., 0., 0.])


c:\Users\23128\anaconda3\envs\venv\lib\site-packages\HTorch\optimizers\radam.py:145: UserWarning: This overload of add_ is deprecated:
	add_(Number alpha, Tensor other)
Consider using one of the following signatures instead:
	add_(Tensor other, *, Number alpha) (Triggered internally at ..\torch\csrc\utils\python_arg_parser.cpp:1420.)
  grad.add_(weight_decay, point)


mch_param after updates Parameter containing:
Parameter(MCHParameter([0.2000, 0.1000, 1.0247], requires_grad=True)), nc=3, manifold=Lorentz, curvature=-1

h_dist (loss) tensor(0., grad_fn=<DistBackward0>)
h_param.grad tensor([0., 0., 0.])
h_param after updates Hyperbolic Parameter containing:
Parameter(HParameter([0.2000, 0.1000, 1.0247], requires_grad=True)), manifold=Lorentz, curvature=-1


In [5]:
mch_x = MCHTensor([0.2, 0.1, 0.3], manifold='Lorentz', nc=3, curvature=-1)
mch_y = MCHTensor([0.5, 0.2, 0.6], manifold='Lorentz', nc=3, curvature=-1)
mch_param = MCHParameter(mch_x, manifold='Lorentz')
print('mch_param before updates', mch_param)
torch.autograd.set_detect_anomaly(True)
rsgd_mch = RiemannianSGD([mch_param], lr=1e-2)
# mch_dist = mch_param.Hdist(mch_param.origin(2, 1, ()))
mch_dist = mch_param.Hdist(mch_y)
print('mch_dist (loss)', mch_dist)
mch_dist.backward()
print('mch_param.grad', mch_param.grad)
rsgd_mch.step()
print('mch_param after updates', mch_param)
print()

h_x = HTensor([0.2, 0.1, 0.3], manifold='Lorentz', curvature=-1)
h_y = HTensor([0.5, 0.2, 0.6], manifold='Lorentz', curvature=-1)
h_param = HParameter(h_x, manifold='Lorentz')
rsgd_h = RiemannianSGD([h_param], lr=1e-2)
h_dist = h_param.Hdist(h_y)
h_dist.backward()
print('h_dist (loss)', h_dist)
print('h_param.grad', h_param.grad)
rsgd_h.step()
print('h_param after updates', h_param)



mch_param before updates Parameter containing:
Parameter(MCHParameter([0.2000, 0.1000, 0.3000], requires_grad=True)), nc=3, manifold=Lorentz, curvature=-1
mch_dist (loss) tensor([0.0005], grad_fn=<AliasBackward0>)
mch_param.grad tensor([0., 0., 0.])
mch_param after updates Parameter containing:
Parameter(MCHParameter([0.2000, 0.1000, 1.0247], requires_grad=True)), nc=3, manifold=Lorentz, curvature=-1

h_dist (loss) tensor([0.0005], grad_fn=<AliasBackward0>)
h_param.grad tensor([0., 0., 0.])
h_param after updates Hyperbolic Parameter containing:
Parameter(HParameter([0.2000, 0.1000, 1.0247], requires_grad=True)), manifold=Lorentz, curvature=-1
